In [70]:
import pandas as pd

data_df = pd.read_csv("https://function-app-test3.azurewebsites.net/api/get?action=raids")
users_df = pd.read_csv("https://function-app-test3.azurewebsites.net/api/get?action=users")

In [100]:
from datetime import timedelta
data = data_df.copy()
users = users_df.copy()

data["message_timestamp"] = pd.to_datetime(data["message_timestamp"], utc=True)
data = data.sort_values("message_timestamp")
data = data[abs(data['message_timestamp'].shift(1) - data['message_timestamp']) > timedelta(seconds=1)].reset_index(drop=True)

users = users.set_index("id")

In [148]:
import os

graph = {}

def add_edge(from_, to):
  try:
    graph[from_][to] += 1
  except KeyError:
    try:
      graph[from_][to] = 1
    except KeyError:
      graph[from_] = {to: 1}


for i, row in data.iterrows():
  try:
    from_name = users.loc[row["from"], "name"]
  except KeyError:
    from_name = row["from"]

  try:
    to_name = users.loc[row["to"], "name"]
  except KeyError:
    to_name = row["to"]

  add_edge(from_name, to_name)


directory = "markdown"

if not os.path.exists(directory):
    os.makedirs(directory)

for user, to in graph.items():
  with open(os.path.join(directory, f"{user}.md"), "w") as f:
    txt = ""
    for to_user, weight in to.items():
      if (weight > 1):
        txt += f"[[{to_user}]] ({weight})\n"
    f.write(txt)


import shutil
shutil.make_archive('graph-markdown', 'zip', 'markdown')

'/content/graph-markdown.zip'